## 1. Building Custom Distribution of LLamaStack

In [134]:
import os
import subprocess
import logging
import requests

import pprint

from llama_stack.distribution.library_client import LlamaStackAsLibraryClient


In [135]:
# llama stack build --image-type venv --config llama_stack/templates/lm-eval/lm-eval-build.yaml
client = LlamaStackAsLibraryClient(".stack-run.yaml")
_ = client.initialize()

Using config .stack-run.yaml:

apis:
- inference
- agents
- vector_io
- safety
- eval
- datasetio
- scoring
- telemetry
- tool_runtime
benchmarks: []
container_image: null
datasets: []
image_name: /Users/chrxu/Documents/llama-stack/.stack
logging: null
metadata_store: null
models: []
providers:
  agents:
  - config:
      persistence_store:
        db_path: /Users/chrxu/.llama/distributions//Users/chrxu/Documents/llama-stack/.stack/agents_store.db
        namespace: null
        type: sqlite
    provider_id: meta-reference
    provider_type: inline::meta-reference
  datasetio:
  - config:
      kvstore:
        db_path: 
/Users/chrxu/.llama/distributions//Users/chrxu/Documents/llama-stack/.stack/huggingface_datasetio.db
        namespace: null
        type: sqlite
    provider_id: huggingface-0
    provider_type: remote::huggingface
  - config:
      kvstore:
        db_path: /Users/chrxu/.llama/distributions//Users/chrxu/Documents/llama-stack/.stack/localfs_datasetio.db
        namespace: null
        type: sqlite
    provider_id: localfs-1
    provider_type: inline::localfs
  eval:
  - config:
      kvstore:
        db_path: /Users/chrxu/.llama/distributions//Users/chrxu/Documents/llama-stack/.stack/meta_reference_eval.db
        namespace: null
        type: sqlite
    provider_id: meta-reference-0
    provider_type: inline::meta-reference
  - config:
      base_url: https://vllm-test.apps.rosa.p2i7w2k6p6w7t7e.3emk.p3.openshiftapps.com/v1/completions
      namespace: test
      use_k8s: true
    provider_id: lmeval-1
    provider_type: remote::lmeval
  inference:
  - config:
      api_token: '********'
      max_tokens: '4096'
      tls_verify: 'true'
      url: https://vllm-test.apps.rosa.p2i7w2k6p6w7t7e.3emk.p3.openshiftapps.com/v1/completions
    provider_id: vllm-0
    provider_type: remote::vllm
  - config: {}
    provider_id: sentence-transformers-1
    provider_type: inline::sentence-transformers
  safety:
  - config:
      excluded_categories: []
    provider_id: llama-guard
    provider_type: inline::llama-guard
  scoring:
  - config: {}
    provider_id: basic-0
    provider_type: inline::basic
  - config: {}
    provider_id: llm-as-judge-1
    provider_type: inline::llm-as-judge
  - config:
      openai_api_key: '********'
    provider_id: braintrust-2
    provider_type: inline::braintrust
  telemetry:
  - config:
      sinks: sqlite
      sqlite_db_path: /Users/chrxu/.llama/distributions//Users/chrxu/Documents/llama-stack/.stack/trace_store.db
    provider_id: meta-reference
    provider_type: inline::meta-reference
  tool_runtime:
  - config:
      api_key: '********'
      max_results: 3
    provider_id: brave-search-0
    provider_type: remote::brave-search
  - config:
      api_key: '********'
      max_results: 3
    provider_id: tavily-search-1
    provider_type: remote::tavily-search
  - config: {}
    provider_id: code-interpreter-2
    provider_type: inline::code-interpreter
  - config: {}
    provider_id: rag-runtime-3
    provider_type: inline::rag-runtime
  - config: {}
    provider_id: model-context-protocol-4
    provider_type: remote::model-context-protocol
  - config:
      api_key: '********'
    provider_id: wolfram-alpha-5
    provider_type: remote::wolfram-alpha
  vector_io:
  - config:
      kvstore:
        db_path: /Users/chrxu/.llama/distributions//Users/chrxu/Documents/llama-stack/.stack/faiss_store.db
        namespace: null
        type: sqlite
    provider_id: faiss
    provider_type: inline::faiss
scoring_fns: []
server:
  auth: null
  port: 8321
  tls_certfile: null
  tls_keyfile: null
shields: []
tool_groups: []
vector_dbs: []
version: '2'

In [136]:
def create_http_client():
    from llama_stack_client import LlamaStackClient
    return LlamaStackClient(base_url="http://localhost:8321")

client = create_http_client()

In [137]:
# sanity check for lmeval as a registered provider
providers = client.providers.list()
for provider in providers:
    if provider.api == "eval":
        print(f"Provider: {provider.provider_id}")
        print()

Provider: meta-reference-0

Provider: lmeval-1



## 2. Run LM-Eval

In [138]:
url = "http://0.0.0.0:8321/v1/eval"

response = requests.get(f"{url}/benchmarks")

if response.status_code == 200:
   pprint.pprint(response.json())

{'data': [{'dataset_id': 'lmeval::mmlu',
           'identifier': 'lmeval::mmlu',
           'metadata': {},
           'provider_id': 'lmeval-1',
           'provider_resource_id': 'string',
           'scoring_functions': ['string'],
           'type': 'benchmark'},
          {'dataset_id': 'lmeval:mmlu',
           'identifier': 'lmeval:mmlu',
           'metadata': {},
           'provider_id': 'lmeval-1',
           'provider_resource_id': 'string',
           'scoring_functions': ['string'],
           'type': 'benchmark'}]}


In [139]:
data = {
    "benchmark_id": "lmeval::mmlu",
    "dataset_id": "lmeval::mmlu",
    "scoring_functions": ["string"],
    "provider_benchmark_id": "string",
    "provider_id": "lmeval-1"
}
response = requests.post(f"{url}/benchmarks", json=data)

In [140]:
# Make the GET request
response = requests.get(f"{url}/benchmarks")
pprint.pprint(response.json())

{'data': [{'dataset_id': 'lmeval::mmlu',
           'identifier': 'lmeval::mmlu',
           'metadata': {},
           'provider_id': 'lmeval-1',
           'provider_resource_id': 'string',
           'scoring_functions': ['string'],
           'type': 'benchmark'},
          {'dataset_id': 'lmeval:mmlu',
           'identifier': 'lmeval:mmlu',
           'metadata': {},
           'provider_id': 'lmeval-1',
           'provider_resource_id': 'string',
           'scoring_functions': ['string'],
           'type': 'benchmark'}]}


In [141]:
logger = logging.getLogger(__name__)

def setup_cluster():
    env = os.environ.copy()
    commands = [
        [
            "oc",
            "apply",
            "-f",
            "https://raw.githubusercontent.com/ruivieira/lls-lmeval-reference/main/manifests/lmeval-sa.yaml"
        ],
        [
            "oc",
            "apply",
            "-f",
            "https://raw.githubusercontent.com/ruivieira/lls-lmeval-reference/main/manifests/lmeval-rbac.yaml"
        ],

    ]
    logger.info("Setting up roles...")
    for command in commands:
        p = subprocess.Popen(args=command, env=env)
        p.wait()

setup_cluster()

serviceaccount/lmeval-sa unchanged
clusterrole.rbac.authorization.k8s.io/lmeval-role unchanged
clusterrolebinding.rbac.authorization.k8s.io/lmeval-rolebinding unchanged
role.rbac.authorization.k8s.io/lmeval-role unchanged
rolebinding.rbac.authorization.k8s.io/lmeval-rolebinding unchanged
role.rbac.authorization.k8s.io/lmeval-role unchanged
rolebinding.rbac.authorization.k8s.io/lmeval-rolebinding unchanged


In [142]:
def enable_online_mode():
    env = os.environ.copy()
    commands = [
        [
            "oc",
            "patch",
            "configmap",
            "trustyai-service-operator-config",
            "-n",
            "redhat-ods-applications",
            "--type",
            "merge",
            "-p",
            '{"data":{"lmes-allow-online":"true","lmes-allow-code-execution":"true"}}'
        ],
        [
           "oc",
           "rollout",
            "restart",
            "deployment",
            "trustyai-service-operator-controller-manager",
            "-n",
            "redhat-ods-applications"
        ],

    ]
    logger.info("Patching the TrustyAI configmap...")
    for command in commands:
        p = subprocess.Popen(args=command, env=env)
        p.wait()

enable_online_mode()

configmap/trustyai-service-operator-config patched (no change)
deployment.apps/trustyai-service-operator-controller-manager restarted


In [143]:
data = {
    "benchmark_id": "lmeval-1::mmlu",
    "benchmark_config": {
        "eval_candidate": {
            "type": "model",
            "model": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
            "provider_id": "lmeval-1",
            "sampling_params": {
                "temperature": 0.7,
                "top_p": 0.9,
                "max_tokens": 256
            },
        "tokenized_"
        "task_name": "mmlu",
        },
    },

}

response = requests.post(f"{url}/benchmarks/lmeval::mmlu/jobs", json=data)
job_id = response.json()['job_id']
print(f"Job ID: {job_id}")

Job ID: lmeval-job-3


## 3. Get Job Results

In [144]:
response = requests.get(f"{url}/benchmarks/lmeval::mmlu/jobs/lmeval-job-3/result")
print(response.json())

{'generations': [], 'scores': {}}


In [ ]:
# clean up
response = requests.delete(f"{url}/benchmarks/lmeval::mmlu/jobs/{job_id}")